In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas import json_normalize
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = "1UPBDV2AHTPBWVQXPY0QMT5PRVZU2353V41XFCAX1JS0EA5H" #My Foursquare client ID
CLIENT_SECRET = "QYBZFUD3LJHKVCPOELBYWI5ERNCX4FJTBVRFHDT4YDGN2JVB" #Your Foursquare Client Secret
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 1UPBDV2AHTPBWVQXPY0QMT5PRVZU2353V41XFCAX1JS0EA5H


In [3]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", "Newark, NJ", "Boston, MA", "Washington D.C.", "Baltimore, MD"]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4d4ae6fc7a7b7dea34424761") # Fried Chicken Joint CATEGORY ID
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of fried chicken joints in {city} = ", results[city]['response']['totalResults'])

Total number of fried chicken joints in New York, NY =  247
Total number of fried chicken joints in Newark, NJ =  37
Total number of fried chicken joints in Boston, MA =  70
Total number of fried chicken joints in Washington D.C. =  104
Total number of fried chicken joints in Baltimore, MD =  96


In [6]:
maps[cities[0]]

In [7]:
maps[cities[1]]

In [8]:
maps[cities[2]]

In [9]:
maps[cities[3]]

In [10]:
maps[cities[4]]

From this data, we can see that New York, Newark and Boston have the best density of fried chicken joints. The next section will analyze this data and get the mean location data.

In [11]:
maps = {}
means = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    means[city] = np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values))
    print(city)
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
0.03233821025887633
Newark, NJ
0.022351513002362686
Boston, MA
0.0330119757311136
Washington D.C.
0.029231510291240895
Baltimore, MD
0.05517107634421137


In [12]:
maps[cities[0]]

In [13]:
maps[cities[1]]

In [14]:
maps[cities[2]]

In [15]:
maps[cities[3]]

In [16]:
maps[cities[4]]

Surprisingly, even though having the fewest number of fried chicken restaurants, Newark is the best place to go on this adventure. One would expect that since Newark has one of the 3 largest airports in the New York metropolitan area, a majority of the restaurants would be in the airport. Looking furher into this analysis, only one restaurant is actually in the airport (World Fried Chicken) strengthening Newark's position.

One curious observation is that the mean distance for Philadelphia is quite high. The comparison between Baltimore and the other cities is below.

In [17]:
for city in cities[0:4]:
    title_string = city + " -> " + cities[4]
    print(title_string)
    print(means[cities[4]] - means[city])

New York, NY -> Baltimore, MD
0.022832866085335038
Newark, NJ -> Baltimore, MD
0.032819563341848684
Boston, MA -> Baltimore, MD
0.022159100613097768
Washington D.C. -> Baltimore, MD
0.025939566052970472


The above analysis shows that there is a difference in mean distance between Baltimore and the other cities of a mininum of two tenths of a mile, further confirming that Baltimore is 5th on the list.

Another observation is that New York, Boston and Washington D.C. have a very small difference in the mean distance. The previous maps show that the aforementioned cities have outlier cities, most likely skewing the data. To better determine the next best city to go to for the fried chicken adventure, the first, two and five largest distances are excluded from the data and a new mean is calculated. This analysis is conducted below.

In [18]:
print("Excluding First Largest:")
for i in range(0,4):
    if i != 1:
        venues_mean_coor = [df_venues[cities[i]]['Lat'].mean(), df_venues[cities[i]]['Lng'].mean()] 

        print(cities[i])
        print("Mean Distance from Mean coordinates")
        dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[cities[i]][['Lat','Lng']].values)
        dists.sort()
        print(np.mean(dists[:-1])) #Ignore the first largest distances

print("\n\nExcluding Two Largest:")
for i in range(0,4):
    if i != 1:
        venues_mean_coor = [df_venues[cities[i]]['Lat'].mean(), df_venues[cities[i]]['Lng'].mean()] 

        print(cities[i])
        print("Mean Distance from Mean coordinates")
        dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[cities[i]][['Lat','Lng']].values)
        dists.sort()
        print(np.mean(dists[:-2])) #Ignore the two largest distances
        
print("\n\nExcluding Five Largest:")
for i in range(0,4):
    if i != 1:
        venues_mean_coor = [df_venues[cities[i]]['Lat'].mean(), df_venues[cities[i]]['Lng'].mean()] 

        print(cities[i])
        print("Mean Distance from Mean coordinates")
        dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[cities[i]][['Lat','Lng']].values)
        dists.sort()
        print(np.mean(dists[:-5])) #Ignore the five largest distances
        
print("\n\nExcluding Ten Largest:")
for i in range(0,4):
    if i != 1:
        venues_mean_coor = [df_venues[cities[i]]['Lat'].mean(), df_venues[cities[i]]['Lng'].mean()] 

        print(cities[i])
        print("Mean Distance from Mean coordinates")
        dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[cities[i]][['Lat','Lng']].values)
        dists.sort()
        print(np.mean(dists[:-10])) #Ignore the five largest distances

Excluding First Largest:
New York, NY
Mean Distance from Mean coordinates
0.031489752793443064
Boston, MA
Mean Distance from Mean coordinates
0.031854490103607275
Washington D.C.
Mean Distance from Mean coordinates
0.02869568478847445


Excluding Two Largest:
New York, NY
Mean Distance from Mean coordinates
0.030776443731933348
Boston, MA
Mean Distance from Mean coordinates
0.030664057852989646
Washington D.C.
Mean Distance from Mean coordinates
0.028179070188624217


Excluding Five Largest:
New York, NY
Mean Distance from Mean coordinates
0.028879082311898133
Boston, MA
Mean Distance from Mean coordinates
0.028127506808964517
Washington D.C.
Mean Distance from Mean coordinates
0.026882701223244754


Excluding Ten Largest:
New York, NY
Mean Distance from Mean coordinates
0.026663453009148505
Boston, MA
Mean Distance from Mean coordinates
0.0253103029959696
Washington D.C.
Mean Distance from Mean coordinates
0.02481902290995265


After analysis, Boston is the next best city to visit followed by New York and and Washington D.C. It should be noted that excluding just one distance would make New York the next best city. However, excluding the top two, five and ten largest yield consistent results.

### Final Ranking
1. Newark, NJ
2. Boston, MA
3. New York, NY
4. Washington, D.C.
5. Baltimore, MD